In [ ]:
pip install syft==0.2.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch as th

In [ ]:
df= pd.read_csv("drive/MyDrive/sample.csv")
input= df.drop(columns= ['id','max hr','intensity'])
input= th.tensor(df.drop(columns= ['id','intensity']).values, dtype=th.float,requires_grad=True)
target= df['intensity']
target= th.tensor(df.drop(columns= ['id','age', 'max hr', 'max hr of session']).values, dtype=th.float, requires_grad=True)

test_input= input[0:20]
test_target= target[0:20]

In [ ]:
import syft as sy
import torch as th
hook= sy.TorchHook(th)
from torch import nn, optim

In [ ]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

#send data to workers

bobs_input = input[21:60].send(bob)
bobs_target = target[21:60].send(bob)


alices_input = input[61:100].send(alice)
alices_target = target[61:100].send(alice)


In [ ]:
# Iniitalize A Linear Model
model = nn.Linear(3,3)

In [ ]:
#20-80 split

iteration = 1
worker_iters = 2

for a_iter in range(iteration):
  
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.01)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.01)


    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_input)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_input)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()
  
        alices_opt.step()
        alices_loss = alices_loss.get().data

    # move model to secure worker
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    # fed avg weight and bias for alice and bob models
    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))
   

Bob:tensor(1.0565e+15) Alice:tensor(1.2234e+15)


In [ ]:
iteration = 1
opt = optim.SGD(params=model.parameters(), lr=0.01)

for wi in range(iteration):
        # Test Model
        opt.zero_grad()
        preds = model(test_input)
        loss = ((preds - test_target) ** 2).sum()
        loss.backward()
        opt.step()
        print(loss)



tensor(1.3372e+24, grad_fn=<SumBackward0>)


In [17]:
bobs_mae= 1.06/40
alices_mae= 1.22/40
nn_mae= 1.34/20
print(bobs_mae)
print(alices_mae)
print(nn_mae)

0.026500000000000003
0.0305
0.067
